# Caso Práctico (tips) - ENGIE
*Jose Enrique Zafra Mena*

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
import json 

## Carga de los datos

In [40]:
facturas_df = pd.read_excel('datos/datos-tips/AGIkey_facturas.xlsx')
mapping_df = pd.read_excel('datos/Mapping.xlsx')
IAC_df = pd.read_excel('datos/datos-tips/MEDG2_IAC_GETRA_ABRIL_2024_06_18_13_34.xlsx')

In [41]:
facturas_df.head(1000)

,NumeroFactura,Emisor,RazonSocialEmisor,DepEmisor,Receptor,RazonSocialReceptor,DepReceptor,FechaFactura,Importe,Moneda,...,FechaRegistro,Destino,Contrato,Origen,ServicioFacturado,¿Verificada por BO?,Observaciones,Importe (<0 a pagar EEE-Enagas),mes,año
0,1400180000,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"4,56",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
1,1400180001,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"15794,20",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
2,1400180002,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"16380,50",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
3,1400180003,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"1930,22",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
4,1400180004,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"43388,45",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,1400180104,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,28/05/2024,"44751,03",NaN,...,28/05/2024,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
936,1400180105,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,28/05/2024,"11928,65",NaN,...,28/05/2024,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
937,1400180106,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,28/05/2024,"11198,08",NaN,...,28/05/2024,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
938,2024069931,ESA86484292,"ENAGAS GTS, SAU",20.0,ESB82508441,ENGIE ESPAÑA SLU,NaN,28/05/2024,"16141,36",NaN,...,28/05/2024,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN


## Filtrado de datos relevantes

### Facturas
Solo nos interesan ciertas columnas: Factura, Importe, Fecha Factura.

In [42]:
facturas_df = facturas_df.drop_duplicates()

facturas_df = facturas_df[['NumeroFactura', 'Importe', 'FechaFactura', 'Origen','ServicioFacturado']]

# solo queremos algunos valores de Origen: C.I.Almería y Entrada PVB
facturas_filtrado_df = facturas_df[(facturas_df['Origen'].isin(['C.I. Almería'])) & (facturas_df['ServicioFacturado'].isin(['Entrada PVB']))]

# cambiamos el formato del importe
facturas_filtrado_df['Importe'] = facturas_filtrado_df['Importe'].str.replace(',' , '.').astype(float)


facturas_filtrado_df.head(10)

C:\Users\kikek\AppData\Local\Temp\ipykernel_16052\520945557.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facturas_filtrado_df['Importe'] = facturas_filtrado_df['Importe'].str.replace(',' , '.').astype(float)


,NumeroFactura,Importe,FechaFactura,Origen,ServicioFacturado
20,2324001054,537278.06,2024-03-01 00:00:00,C.I. Almería,Entrada PVB
159,2324002755,514862.14,2024-05-02 00:00:00,C.I. Almería,Entrada PVB
376,2324004231,503763.47,2024-06-03 00:00:00,C.I. Almería,Entrada PVB
570,2324005598,536445.62,2024-04-04 00:00:00,C.I. Almería,Entrada PVB
771,2324007434,525023.53,2024-07-05 00:00:00,C.I. Almería,Entrada PVB


### Mapping

In [43]:
mapping_df = mapping_df.drop_duplicates()
print(mapping_df.shape)

# solo nos interesan algunos valores: Commodity == IAC
mapping_filtrado_df = mapping_df[(mapping_df['Commodity'].isin(['IAC']))]
mapping_filtrado_df.head(200)

(29, 5)


,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA


### Deals 
Solo queremos las columnas: Deal, Cantidad, Start-DeliveryDate, End-DeliveryDate, Procedencia (SUGST_AOC_STOK), Subflow 1, Date 1, Subcantidad 1, Subflow 2, Date 2, Subcantidad 2.

En la columna flows vemos que hay datos en json. Tratamos esa columna para sacar los datos a un formato mejor. También mantemos el Id del deal para cada subflow.

In [44]:
IAC_df = IAC_df.drop_duplicates()

# nos quedamos con las columnas que queremos por ahora
IAC_filtrado_df = IAC_df[['Id', 'Flows', 'StartDeliveryDate', 'EndDeliveryDate']]
IAC_filtrado_df = IAC_filtrado_df.rename(columns={'Id':'Id Deal'})

IAC_filtrado_df.head()

,Id Deal,Flows,StartDeliveryDate,EndDeliveryDate
0,37591650,"[{'Currency': 'EUR', 'Amount': 105279.586144, ...",2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00
1,37056084,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 9219...",2023-01-01T00:00:00+01:00,2023-12-31T00:00:00+01:00


In [45]:
# definimos una función para tratar los json (la tendremos que usar varias veces)
def clean_json(string):
    if isinstance(string, str):
        string = string.replace("'", '"')
        string = string.replace("None", 'null')
        string = string.replace("True", 'true')
        string = string.replace("False", 'false')
    return string

# la usamos sobre la columna de flows
flows_cleaned = IAC_filtrado_df['Flows'].apply(clean_json)


# Ahora creamos otra función para extraer los datos de json a diccionarios de python
def extract_json(flows, id_deal):
    if not flows or not isinstance(flows, list):
        return pd.DataFrame()
    data = {
        'Id Deal': [],
        'Currency': [],
        'Amount': [],
        'Id': [],
        'Date': [],
        'FlowType': [],
        'PayReceive': []
    }
    for item in flows:
        if isinstance(item, dict):
            data['Id Deal'].append(id_deal)
            for key in data.keys():
                if key != 'Id Deal':
                    data[key].append(item.get(key))
        else:
            print(f"Unexpected type {type(item)}")
    return pd.DataFrame(data)

# creamos un df para meter los datos
datos_json = []

for idx, row in IAC_filtrado_df.iterrows():
    try:
        # intentamos convertir la cadena a una lista de diccionarios
        print(f"Procesamos fila {idx} con Id Deal {row['Id Deal']}")
        flows = eval(row['Flows'])
        id_deal = row['Id Deal']
        i = extract_json(flows, id_deal)
        if not i.empty:
            datos_json.append(i)
        else:
            print(f"DataFrame vacío para Id Deal {id_deal}")
    except Exception as e:
        print(e)

datos_json_final = pd.concat(datos_json, ignore_index=True)
    
# cambiamos un poco las columnas
datos_json_final = datos_json_final[['Id Deal','Id','Amount', 'Date', 'FlowType']]
datos_json_final = datos_json_final.rename(columns={'Id':'Id Flow', 'Amount':'Importe Flow', 'Date':'Fecha Flow', 'FlowType':'Subflow'})

datos_json_final.head(100)

Procesamos fila 0 con Id Deal 37591650
Procesamos fila 1 con Id Deal 37056084


,Id Deal,Id Flow,Importe Flow,Fecha Flow,Subflow
0,37591650,93540225,105279.586144,2024-02-20T00:00:00+01:00,1
1,37591650,93540224,409582.490000,2024-02-20T00:00:00+01:00,2
2,37591650,93540227,120318.957648,2024-03-20T00:00:00+01:00,1
3,37591650,93540226,383444.510000,2024-03-20T00:00:00+01:00,2
4,37591650,93540229,128084.859614,2024-04-22T00:00:00+02:00,1
5,37591650,93540228,408200.220000,2024-04-22T00:00:00+02:00,2
6,37591650,93540231,130004.564430,2024-05-20T00:00:00+02:00,1
7,37591650,93540230,395032.480000,2024-05-20T00:00:00+02:00,2
8,37591650,93540233,134338.049911,2024-06-20T00:00:00+02:00,1
9,37591650,93540232,408200.220000,2024-06-20T00:00:00+02:00,2


Lo anterior solo es la columna que teníamos de flows. Ahora lo unimos con el resto del dataframe original (deals IAC). Tan solo tenemos que añadirle las columnas de las fechas iniciales y finales, según el valor de Id Deal que tenga.

In [46]:
IAC_final_df = (pd.merge(IAC_filtrado_df, datos_json_final, on='Id Deal', how='inner')).drop_duplicates()
IAC_final_df = IAC_final_df.drop(columns=['Flows'])

# también añadimos una columna llamada Commodity con valor IAC para que sea más fácil el merge
IAC_final_df['Commodity']='IAC'

IAC_final_df.head(100)
## podemos cambiar el formato de las fechas (por ahora no??) 

,Id Deal,StartDeliveryDate,EndDeliveryDate,Id Flow,Importe Flow,Fecha Flow,Subflow,Commodity
0,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540225,105279.586144,2024-02-20T00:00:00+01:00,1,IAC
1,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540224,409582.490000,2024-02-20T00:00:00+01:00,2,IAC
2,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540227,120318.957648,2024-03-20T00:00:00+01:00,1,IAC
3,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540226,383444.510000,2024-03-20T00:00:00+01:00,2,IAC
4,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540229,128084.859614,2024-04-22T00:00:00+02:00,1,IAC
5,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540228,408200.220000,2024-04-22T00:00:00+02:00,2,IAC
6,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540231,130004.564430,2024-05-20T00:00:00+02:00,1,IAC
7,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540230,395032.480000,2024-05-20T00:00:00+02:00,2,IAC
8,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540233,134338.049911,2024-06-20T00:00:00+02:00,1,IAC
9,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540232,408200.220000,2024-06-20T00:00:00+02:00,2,IAC


### Merge de Mapping con Deals
Hacemos el merge con Commodity

In [47]:
IAC_merged_df = pd.merge(IAC_final_df, mapping_filtrado_df, on='Commodity', how='inner')

# quitamos algunas columnas innecesarias
IAC_merged_df.drop(columns=['Portfolio','DealType'], inplace=True)


IAC_merged_df.head(50)

,Id Deal,StartDeliveryDate,EndDeliveryDate,Id Flow,Importe Flow,Fecha Flow,Subflow,Commodity,Origen,Servicio facturado
0,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540225,105279.586144,2024-02-20T00:00:00+01:00,1,IAC,C.I. Almería,Entrada PVB
1,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540224,409582.490000,2024-02-20T00:00:00+01:00,2,IAC,C.I. Almería,Entrada PVB
2,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540227,120318.957648,2024-03-20T00:00:00+01:00,1,IAC,C.I. Almería,Entrada PVB
3,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540226,383444.510000,2024-03-20T00:00:00+01:00,2,IAC,C.I. Almería,Entrada PVB
4,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540229,128084.859614,2024-04-22T00:00:00+02:00,1,IAC,C.I. Almería,Entrada PVB
5,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540228,408200.220000,2024-04-22T00:00:00+02:00,2,IAC,C.I. Almería,Entrada PVB
6,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540231,130004.564430,2024-05-20T00:00:00+02:00,1,IAC,C.I. Almería,Entrada PVB
7,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540230,395032.480000,2024-05-20T00:00:00+02:00,2,IAC,C.I. Almería,Entrada PVB
8,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540233,134338.049911,2024-06-20T00:00:00+02:00,1,IAC,C.I. Almería,Entrada PVB
9,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540232,408200.220000,2024-06-20T00:00:00+02:00,2,IAC,C.I. Almería,Entrada PVB


### Merge de Mapping con Facturas
En este caso nos vale bien con Origen, o bien con Servicio Facturado.

In [49]:
facturas_merged_df = pd.merge(facturas_filtrado_df, mapping_filtrado_df, on='Origen', how='inner')


facturas_merged_df = facturas_merged_df.rename(columns={'ServicioFacturado':'Servicio facturado'})
facturas_merged_df = facturas_merged_df.loc[:, ~facturas_merged_df.columns.duplicated()]

facturas_merged_df.head(100)

,NumeroFactura,Importe,FechaFactura,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,2324001054,537278.06,2024-03-01 00:00:00,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA
1,2324002755,514862.14,2024-05-02 00:00:00,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA
2,2324004231,503763.47,2024-06-03 00:00:00,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA
3,2324005598,536445.62,2024-04-04 00:00:00,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA
4,2324007434,525023.53,2024-07-05 00:00:00,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA


## Conciliación
**Se hace según *Origen* + *Servicio Facturado***

Mapping y Facturas comparten *Origen* y *Servicio Facturado*

Vemos la cantidad según Facturas, la cantidad según Deals, y hacemos la diferencia

### Comparación entre Facturas y Deals


In [50]:
# agrupamos y sumamos importes
IAC_grouped = IAC_merged_df.groupby(['Id Deal','Origen', 'Servicio facturado', 'Subflow'])['Importe Flow'].sum()

IAC_grouped.head(100)

Id Deal   Origen        Servicio facturado  Subflow
37056084  C.I. Almería  Entrada PVB         1          6.982081e+05
                                            2          4.264376e+06
37591650  C.I. Almería  Entrada PVB         1          7.051156e+05
                                            2          5.115923e+06
Name: Importe Flow, dtype: float64

In [53]:
# hacemos lo mismo para facturas
facturas_grouped = facturas_merged_df.groupby(['NumeroFactura','Origen', 'Servicio facturado'])['Importe'].sum()
facturas_grouped.head(100)

NumeroFactura  Origen        Servicio facturado
2324001054     C.I. Almería  Entrada PVB           537278.06
2324002755     C.I. Almería  Entrada PVB           514862.14
2324004231     C.I. Almería  Entrada PVB           503763.47
2324005598     C.I. Almería  Entrada PVB           536445.62
2324007434     C.I. Almería  Entrada PVB           525023.53
Name: Importe, dtype: float64

### DataFrame de conciliación
Falta poner columnas de Subflows, Numero de Factura, Id Deal

In [54]:
# unimos los dataframes para comparar
conciliacion_df = pd.merge(IAC_grouped, facturas_grouped, on=['Origen', 'Servicio facturado'], how='outer')

# hallamos la diferencia 
conciliacion_df['Diferencia'] = conciliacion_df['Importe'] - conciliacion_df['Importe Flow']  

conciliacion_df.head()

Importe Flow    Importe     Diferencia
Origen       Servicio facturado                                         
C.I. Almería Entrada PVB         698208.056484  537278.06 -160929.996484
             Entrada PVB         698208.056484  514862.14 -183345.916484
             Entrada PVB         698208.056484  503763.47 -194444.586484
             Entrada PVB         698208.056484  536445.62 -161762.436484
             Entrada PVB         698208.056484  525023.53 -173184.526484